In [ ]:
import boost_histogram as bh
import numpy as np
import matplotlib.pyplot as plt

# Using boost-histogram

## 1: Basic 1D histogram

Let's start with the basics. We will create a histogram using boost-histogram and fill it.

### 1.1: Data

Let's make a 1d dataset to run on.

In [ ]:
data1 = np.random.normal(3.5, 2.5, size=1_000_000)

Now, let's make a histogram

In [ ]:
hist1 = bh.histogram(bh.axis.regular(40, -2, 10))

In [ ]:
hist1.fill(data1);

Let's see how many entries are in the histogram:

In [ ]:
hist1.sum()

What happened to the missing items?

In [ ]:
hist1.sum(flow=True)

Like ROOT, we have overflow bins by default. We can turn them off, but they enable some powerful things like projections.

Let's plot this (Hist should make this easier):

In [ ]:
plt.bar(hist1.axis(0).centers, hist1.view(), width=hist1.axis(0).widths);

Note: You can leave off the `view()` if you want to. Also we might add:
        
```python
plt.bar(hist.axes.centers, hist, width=hist.axes.widths);
```

From now on, let's be lazy

In [ ]:
plothist = lambda h: plt.bar(h.axis(0).centers, h.view(), width=h.axis(0).widths);

Aside: here's step. It's quite ugly for us, just like it is for numpy. Or anyone.

In [ ]:
plt.step(hist1.axis(0).edges[:-1], hist1.view(), where='post');

No plotting is built in, but the data is easy to access.

## 2: Drop-in replacement for Numpy

To start using this yourself, you don't even need to change your code. Let's try the numpy adapters.

In [ ]:
bins2, edges2 = bh.numpy.histogram(data1, bins=10)

In [ ]:
b2, e2 = np.histogram(data1, bins=10)

In [ ]:
bins2 - b2

In [ ]:
e2 - edges2

Not bad! Let's start moving to the boost-histogram API, so we can use our little plotting function:

In [ ]:
hist2 = bh.numpy.histogram(data1, bins='auto', bh=True)
plothist(hist2);

Now we can move over to boost-histogram one step at a time! Just to be complete, we can also go the other direction:

In [ ]:
b2p, e2p = bh.numpy.histogram(data1, bins=10, bh=True).to_numpy()
b2p == b2

## 3: More dimensions

The same API works for multiple dimensions.

In [ ]:
hist3 = bh.histogram(
    bh.axis.regular(150, -1.5, 1.5),
    bh.axis.regular(100, -1, 1)
)

In [ ]:
def make_2D_data(*, mean=(0,0), widths=(1,1), size=1_000_000):
    cov = np.asarray(widths) * np.eye(2)
    return np.random.multivariate_normal(mean, cov, size=size).T.copy()

Note: Boost-histogram does not respect the stride information, so you must copy here to normalize to C ordering.

In [ ]:
data3x = make_2D_data(mean=[-.75, .5], widths=[.2, 0.02])
data3y = make_2D_data(mean=[.75, .5], widths=[.2, 0.02])

In [ ]:
hist3.reset()
hist3.fill(*data3x)
hist3.fill(*data3y)

Again, let's factor out into a little function:

In [ ]:
def plothist2d(h):
    X, Y = np.meshgrid(*(a.edges for a in (h.axis(i) for i in range(h.rank))), indexing='ij')
    
    
    # May become
    # X, Y = hist3.axes.edges
    # X, Y = np.broadcast_arrays(X, Y)
    
    return plt.pcolormesh(X, Y, h.view())

In [ ]:
plothist2d(hist3);

Let's try a 3D histogram

In [ ]:
data3d = [np.random.normal(size=1_000_000) for _ in range(3)]

hist3d = bh.histogram(
    bh.axis.regular(150, -5, 5),
    bh.axis.regular(100, -5, 5),
    bh.axis.regular(100, -5, 5)
)

hist3d.fill(*data3d)

In [ ]:
plothist2d(hist3d.project(0,1));

## 4: UHI

Let's explore the boost-histogram UHI syntax. We will reuse the previous 2D histogram from part 3:

In [ ]:
plothist2d(hist3);

I can see that I want y from 0.25 to 0.75, in data coordinates:

In [ ]:
plothist2d(hist3[:, bh.loc(.25):bh.loc(.75)]);

How about a 1d histogram?

WARNING: `bh.project` may change names in next release, possibly will just become `sum`.

In [ ]:
plothist(hist3[:, ::bh.project]);
plothist(hist3[::bh.project, :]);

Let's look at one part and rebin:

In [ ]:
plothist2d(hist3[:50:bh.rebin(2), 50::bh.rebin(2)]);

What is the value at `(-.75, .5)`?

In [ ]:
hist3[bh.loc(-.75), bh.loc(.5)]

## 5: Understanding accumulators

Boost-histogram has several different storages; storages store accumulators. Let's try making a profile.

In [ ]:
mean = bh.accumulators.mean()
mean.fill([.3, .4, .5])

In [ ]:
print(f"mean.count={mean.count} mean.value={mean.value} mean.variance={mean.variance}")

# Python 3.8:
# print(f"{mean.count=} {mean.value=} {mean.variance=}")

## 6: Changing the storage

In [ ]:
hist6 = bh.histogram(bh.axis.regular(10,0,10), storage=bh.storage.mean)

In [ ]:
hist6.fill([0.5]*3, sample=[.3, .4, .5])

In [ ]:
hist6[0]

> In the current beta, the values here are not accessable via `.view()`.

## 7: Making a density histogram

Let's try to make a density histogram like Numpy's.

In [ ]:
bins = [-10, -7, -4, -3, -2, -1, -.75, -.5, -.25, 0, .25, .5, .75, 1, 2, 3, 4, 7, 10]
d7, e7 = np.histogram(data1 - 3.5, bins=bins, density=True)
plt.hist(data1 - 3.5, bins=bins, density=True);

Yes, it's ugly. Don't judge.

Density is not supported yet! What do we do?

In [ ]:
hist7 = bh.numpy.histogram(data1 - 3.5, bins=bins, bh=True)

widths = np.meshgrid(*(a.widths for a in (hist7.axis(i) for i in range(hist7.rank))), indexing='ij', sparse=True)
area = np.prod(widths, axis=0)

# May become
# X = hist3.axes.widths

area

Yes, that does not need to be so complicated for 1D, but it's general.

In [ ]:
factor = np.sum(hist7.view())
view = hist7.view() / (factor * area)

Setting with array support would make this simpler, planned.

In [ ]:
plt.bar(hist7.axis(0).centers, view, width=hist7.axis(0).widths);

# 8: Axis types

In [ ]:
hist8 = bh.histogram(
    bh.axis.regular_log(30, 1,10),
    bh.axis.regular_sqrt(30, 1,10)
)

In [ ]:
hist8.reset()
hist8.fill(*make_2D_data(mean=(5,5), widths=(5,5)))

In [ ]:
plothist2d(hist8);

# 9: And, circular, too!

In [ ]:
hist9 = bh.histogram(bh.axis.circular(30, 0, 2*np.pi))
hist9.fill(np.random.uniform(0, np.pi*4, size=300))

Now, the really complicated part:
    
<!--
ax = plt.subplot(111, polar=True)
plothist(hist9);
-->